## Gold Layer: Business-ready data for visualization

In [ ]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
from pyspark.sql.functions import col, monotonically_increasing_id, year, month, dayofmonth, hour, minute, second, date_format, quarter

StatementMeta(, 122dabc2-3397-4f9f-a088-90d5f5027549, 4, Finished, Available, Finished)

In [ ]:
# Load new data daily
#start_date = "2025-01-15" 
data_df = spark.read.table("events_silver").filter(col('time') > start_date)

StatementMeta(, 122dabc2-3397-4f9f-a088-90d5f5027549, 5, Finished, Available, Finished)

In [ ]:
#data_df.count()

StatementMeta(, 122dabc2-3397-4f9f-a088-90d5f5027549, 6, Finished, Available, Finished)

234

In [ ]:
# Merge Dimensions (Type 1 SCD)
def merge_dimensions(delta_table_path, new_df, primary_key):
    delta_table = DeltaTable.forPath(spark, delta_table_path)
    delta_table.alias("existing").merge(
        new_df.alias("new"),
        f"existing.{primary_key} = new.{primary_key}"
    ).whenMatchedUpdateAll() \
     .whenNotMatchedInsertAll() \
     .execute()

StatementMeta(, 122dabc2-3397-4f9f-a088-90d5f5027549, 7, Finished, Available, Finished)

In [ ]:

Place_Dimension_path = "abfss://dd4e4e02-925b-4769-9c27-f6784f8f60b3@onelake.dfs.fabric.microsoft.com/afc11570-f375-4319-95ea-ded6baa480a8/Tables/place_dimension"

# Update Place Dimension
new_place_df = data_df.select("place", "country_code").distinct()
existing_place_df = spark.read.table("Place_Dimension")
new_place_df = new_place_df.join(existing_place_df, ["place", "country_code"], "left_anti")
new_place_df = new_place_df.withColumn("place_id", monotonically_increasing_id())
merge_dimensions(Place_Dimension_path, new_place_df, "place_id")

StatementMeta(, 122dabc2-3397-4f9f-a088-90d5f5027549, 9, Finished, Available, Finished)

In [ ]:
# Update Magnitude Category Dimension
Mag_Cat_Dimension="abfss://dd4e4e02-925b-4769-9c27-f6784f8f60b3@onelake.dfs.fabric.microsoft.com/afc11570-f375-4319-95ea-ded6baa480a8/Tables/magnitude_category_dimension"

# Update Magnitude Category Dimension
new_mag_cat_df = data_df.select("magCategory", "sig_class", "magType").distinct()
existing_mag_cat_df = spark.read.table("Magnitude_Category_Dimension")
new_mag_cat_df = new_mag_cat_df.join(existing_mag_cat_df, ["magCategory", "sig_class", "magType"], "left_anti")
new_mag_cat_df = new_mag_cat_df.withColumn("magCategory_id", monotonically_increasing_id())
merge_dimensions(Mag_Cat_Dimension, new_mag_cat_df, "magCategory_id")


StatementMeta(, 122dabc2-3397-4f9f-a088-90d5f5027549, 10, Finished, Available, Finished)

In [ ]:
Time_Dimension_path = "abfss://dd4e4e02-925b-4769-9c27-f6784f8f60b3@onelake.dfs.fabric.microsoft.com/afc11570-f375-4319-95ea-ded6baa480a8/Tables/time_dimension"

# Update Time Dimension
new_time_df = data_df.select(
    col("time").alias("full_date").cast("date")
).distinct()
existing_time_df = spark.read.table("Time_Dimension")
new_time_df = new_time_df.join(existing_time_df, "full_date", "left_anti")
new_time_df = new_time_df \
    .withColumn("year", year(col("full_date"))) \
    .withColumn("month", month(col("full_date"))) \
    .withColumn("day", dayofmonth(col("full_date"))) \
    .withColumn("month_name", date_format(col("full_date"), "MMMM")) \
    .withColumn("week_name", date_format(col("full_date"), "EEEE")) \
    .withColumn("quarter", quarter(col("full_date")))
merge_dimensions(Time_Dimension_path, new_time_df, "full_date")


StatementMeta(, 122dabc2-3397-4f9f-a088-90d5f5027549, 8, Finished, Available, Finished)

In [ ]:
# Append to Fact Table
new_fact_df = data_df.withColumn("full_date", col("time").cast("date")) \
    .join(spark.table("Place_Dimension"), ["place", "country_code"], "left") \
    .join(spark.table("Magnitude_Category_Dimension"), ["magCategory", "sig_class", "magType"], "left") \
    .join(spark.table("Time_Dimension"), "full_date", "left") \
    .select(
        col("magnitude"),
        col("latitude"),
        col("longitude"),
        col("depth"),
        col("sig"),
        col("updated"),
        col("place_id"),
        col("magCategory_id"),
        col("full_date")
    )
new_fact_df.write.format("delta").mode("append").saveAsTable("Earthquake_Fact")


StatementMeta(, 122dabc2-3397-4f9f-a088-90d5f5027549, 11, Finished, Available, Finished)